In [1]:
%load_ext autoreload
%autoreload 2

# Standard imports
from pathlib import Path
import sys

# Library imports
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as sc
from sklearn.metrics import mean_squared_error
import xarray as xr

# Util imports
sys.path.append("../../")
import src.create_reference_data as cr

# Correct station data

This notebook applies bias correction algorithms on gridded data using station data.

**Prerequisite**: Run `notebooks/02-apply-bias-correction/01_overlay_station.ipynb`

### Set input parameters

In [3]:
CITY_NAME = "Mandaue"  # with suffix for Cagayan De Oro

DATE = "2008-07-01"  # sample date for debugging
YEARS = [2007, 2008, 2009, 2016, 2017, 2018]
SHOULD_DEBUG = False
PROCESSED_PATH = Path("../../data/02-processed")
CORRECTED_PATH = PROCESSED_PATH / "bias-correction-radial-optimized"
CORRECTED_PATH.mkdir(parents=True, exist_ok=True)

STATION_NC = CORRECTED_PATH / f"station_{CITY_NAME.lower()}.nc"
GRIDDED_NC = (
    PROCESSED_PATH
    / f"input/chirts_chirps_regridded_interpolated_{CITY_NAME.split('_')[0].lower()}.nc"
)
GRIDDED_SUBSET_NC = CORRECTED_PATH / f"gridded_{CITY_NAME.lower()}.nc"

### Set run parameters
- `plot_offset`: margin for x and y bounds

In [4]:
variable_params = dict(
    tmin="CHIRTS minimum temperature",
    tmax="CHIRTS maximum temperature",
    precip="CHIRPS precipitation",
)

method_params = dict(
    tmin="linear_scaling",
    tmax="linear_scaling",
    precip="linear_scaling",
)

algo_params = dict(
    linear_scaling=dict(
        name="Linear Scaling",
        func=cr.correct_gridded_cmethods,
        group="time.month",
    ),
    variance_scaling=dict(
        name="Variance Scaling",
        func=cr.correct_gridded_cmethods,
        group="time.month",
    ),
    quantile_mapping=dict(
        name="Quantile Mapping",
        func=cr.correct_gridded_cmethods,
        n_quantiles=10,
    ),
    detrended_quantile_mapping=dict(
        name="Detrended Quantile Mapping",
        func=cr.correct_gridded_cmethods,
        n_quantiles=1_000,
    ),
)

# refer to defaults
for method, algo_param in algo_params.items():
    if "group" not in algo_param.keys():
        algo_param["group"] = "time.month"
    if "n_quantiles" not in algo_param.keys():
        algo_param["n_quantiles"] = 1_000


scatterplot_params = dict(
    tmin=dict(
        variable_name="Minimum Temperature",
        units="°C",
        plot_offset=2.5,
        color="firebrick",
    ),
    tmax=dict(
        variable_name="Maximum Temperature",
        units="°C",
        plot_offset=1,
        color="firebrick",
    ),
    precip=dict(
        variable_name="Precipitation",
        units="mm/day",
        plot_offset=10,
        color="dodgerblue",
    ),
)

### Load data

In [5]:
station_ds = xr.open_dataset(STATION_NC, engine="scipy")
gridded_ds = xr.open_dataset(GRIDDED_NC, engine="scipy")
gridded_subset_ds = xr.open_dataset(GRIDDED_SUBSET_NC, engine="scipy")

In [6]:
gridded_subset_ds

<xarray.Dataset>
Dimensions:      (time: 7305, lat: 12, lon: 10)
Coordinates:
  * time         (time) datetime64[ns] 2003-01-01 2003-01-02 ... 2022-12-31
  * lat          (lat) float64 10.21 10.23 10.25 10.27 ... 10.39 10.41 10.43
  * lon          (lon) float64 123.9 123.9 123.9 123.9 ... 124.0 124.0 124.1
    band         int32 ...
    spatial_ref  int32 ...
Data variables:
    tmin         (time, lat, lon) float32 ...
    tmax         (time, lat, lon) float32 ...
    precip       (time, lat, lon) float32 ...
Attributes: (12/16)
    Conventions:    CF-1.6
    title:          dataVersion 1.0
    history:        created by Climate Hazards Center
    version:        Version 1.0
    date_created:   2023-05-15
    creator_name:   Pete Peterson
    ...             ...
    website:        https://chc.ucsb.edu/data/chirtsmonthly    https://chc.uc...
    comments:       
    reference-1:    Funk, C. et al. A high-resolution 1983–2016 Tmax climate ...
    reference-2:    Verdin, A. et al. Development and validation of the CHIRT...
    reference-3:    Tuholske, C., Caylor, K., Funk, C., Verdin, A., Sweeney, ...
    regrid_method:  bilinear

In [7]:
station_lat = station_ds["lat"].item()
station_lon = station_ds["lon"].item()

# Apply bias correction

In [8]:
stats_list = []
corrected_ds = xr.Dataset(data_vars=None)
for var, title in variable_params.items():
    print(f"Now doing {title}")

    if var == "precip":
        gridded_da = gridded_ds.isel(time=gridded_ds.time.dt.year.isin(YEARS))[var]
        gridded_subset_da = gridded_subset_ds.isel(
            time=gridded_subset_ds.time.dt.year.isin(YEARS)
        )[var]
        station_da = station_ds.isel(time=station_ds.time.dt.year.isin(YEARS))[var]
    else:
        gridded_da = gridded_ds.isel(time=gridded_ds.time.dt.year.isin(YEARS))[var]  #
        gridded_subset_da = gridded_subset_ds.isel(
            time=gridded_subset_ds.time.dt.year.isin(YEARS)
        )[var]
        station_da = station_ds.isel(time=station_ds.time.dt.year.isin(YEARS))[var]

    if SHOULD_DEBUG:
        gridded_da.sel(time=DATE, method="nearest").plot()
        plt.plot(station_lon, station_lat, "o")
        plt.show()

        gridded_subset_da.sel(time=DATE, method="nearest").plot()
        plt.plot(station_lon, station_lat, "o")
        plt.show()

        gridded_subset_da.plot.hist(bins=15)
        plt.show()

    algo_param = algo_params[method_params[var]]
    print(f"Now doing {algo_param['name']} bias correction")

    if algo_param["name"] == "Li et al. (2019)" or algo_param["name"] == "Z-Score":
        corrected_da = algo_param["func"](
            gridded_subset_da.sel(time=DATE, method="nearest"),
            station_da=station_da.sel(time=DATE, method="nearest"),
            std_scale=0.1,
            should_plot=SHOULD_DEBUG,
        )
    else:
        corrected_da = algo_param["func"](
            gridded_da=gridded_subset_da,
            station_da=station_da[:, 0, 0].drop_vars(["lat", "lon"]),
            method=method_params[var],
            n_quantiles=algo_param["n_quantiles"],
            group=algo_param["group"],
            should_plot=SHOULD_DEBUG,
        )

    corrected_ds[var] = corrected_da

    station_aligned_da, corrected_aligned_da = xr.align(
        station_da.mean(dim=["lat", "lon"], skipna=True).dropna(dim="time"),
        corrected_da.mean(dim=["lat", "lon"], skipna=True).dropna(dim="time"),
        join="inner",
    )

    corr, pval = sc.pearsonr(station_aligned_da, corrected_aligned_da)
    rmse = mean_squared_error(station_aligned_da, corrected_aligned_da, squared=False)

    stats_list.append(
        dict(var=var, method=method_params[var], corr=corr, pval=pval, rmse=rmse)
    )

    if SHOULD_DEBUG:
        gridded_subset_slice_da = gridded_subset_da.sel(time=DATE, method="nearest")
        corrected_slice_da = corrected_da.sel(time=DATE, method="nearest")

        plot_min = min([corrected_slice_da.min(), gridded_subset_slice_da.min()]).values
        plot_max = max([corrected_slice_da.max(), gridded_subset_slice_da.max()]).values

        gridded_subset_slice_da.plot(vmin=plot_min, vmax=plot_max)
        plt.title(title)
        plt.show()

        corrected_slice_da.plot(vmin=plot_min, vmax=plot_max)
        plt.title(f"Corrected {title}\n{algo_param['name']}")
        plt.show()

        abs(corrected_slice_da - gridded_subset_slice_da).plot(cmap="RdYlGn")
        plt.title(
            f"Difference between corrected and uncorrected\n{title}\n{algo_param['name']}"
        )
        plt.show()
stats_df = pd.DataFrame(stats_list)

Now doing CHIRTS minimum temperature
Now doing Linear Scaling bias correction
Now doing CHIRTS maximum temperature
Now doing Linear Scaling bias correction
Now doing CHIRPS precipitation
Now doing Linear Scaling bias correction


/home/jace/miniconda3/envs/climate-analytics/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/jace/miniconda3/envs/climate-analytics/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/jace/miniconda3/envs/climate-analytics/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [9]:
corrected_ds.to_netcdf(
    CORRECTED_PATH / f"corrected_subset_{CITY_NAME.lower()}.nc",
    engine="scipy",
)

In [10]:
stats_df.to_parquet(CORRECTED_PATH / f"stats_{CITY_NAME.lower()}.parquet")
stats_df

,var,method,corr,pval,rmse
0,tmin,linear_scaling,0.608023,1.618519e-148,0.820709
1,tmax,linear_scaling,0.804441,0.000000e+00,0.927073
2,precip,linear_scaling,0.486351,1.572222e-130,10.419902
